In [73]:
import random
import numpy as np

### **Задача 1. «Однорукий бандит»**

Пусть имеется игровой автомат с ручкой, с помощью которой его запускают
для игры – «однорукий бандит». Предположим, что игра ведётся по
следующей схеме:
1) В начальный момент времени у игрока есть одна монета
2) Вероятность выигрыша при запуске игры – 0.5. При выигрыше игрок
получает 1 монету. При проигрыше, соответственно, теряет 1 монету.
3) Игра заканчивается либо если игрок обанкротится (поражение), либо
если выиграет 10 монет (победа).

Определить вероятность выигрыша. Также оценить среднее число шагов,
требующееся для победы и для поражения.

In [76]:
def simulate_game():
    """Симулирует одну игру"""
    coins = 1
    steps = 0
    while coins and coins != 10:
        game_result = 1 if random.random() < 0.5 else 0
        if game_result:
            coins += 1
        else:
            coins -= 1
        steps += 1
    return coins == 10, steps

In [78]:
def monte_carlo_games_simulation(num_simulations):
    """Проводит серию симуляций игр методом Монте-Карло"""
    wins = 0
    total_steps_win = 0 
    total_steps_loss = 0 
    for _ in range(num_simulations):
        win, steps = simulate_game()
        if win:
            total_steps_win += steps
            wins += 1
        else:
            total_steps_loss += steps
    wins_probability = wins / num_simulations
    losses = num_simulations - wins
    average_steps_win = total_steps_win / wins if wins > 0 else 0
    average_steps_loss = total_steps_loss / losses if losses > 0 else 0
    return wins_probability, average_steps_win, average_steps_loss

In [80]:
num_simulations = 1_000_000

In [82]:
wins_probability, average_steps_win, average_steps_loss = monte_carlo_games_simulation(num_simulations)

In [83]:
print("ОТВЕТ:")
print(f"Вероятность выигрыша: {wins_probability:.4f}")
print(f"Среднее число шагов, требующееся для победы: {average_steps_win:.2f}")
print(f"Среднее число шагов, требующееся для поражения: {average_steps_loss:.2f}")

ОТВЕТ:
Вероятность выигрыша: 0.0997
Среднее число шагов, требующееся для победы: 33.03
Среднее число шагов, требующееся для поражения: 6.31


### **Задача 2. Распределение зерен по шахматной доске.**
На шахматную доску случайным образом бросают 64 зерна. Определить, как
зерна по количеству распределятся в клетках – то есть, сколько в среднем
клеток будет содержать N зёрен для каждого N.

In [87]:
def simulate_throw():
    """Симулирует один бросок"""
    board = np.zeros(64, dtype=int)
    for _ in range(64):
        grain = random.randint(0, 63)
        board[grain] += 1
    grain_distribution = np.zeros(64, dtype=int)
    for quantity in board:
        grain_distribution[quantity] += 1
    return grain_distribution

In [89]:
def monte_carlo_throws_simulation(num_simulations):
    """Проводит серию симуляций бросков методом Монте-Карло"""
    total_grain_distribution = np.zeros(64, dtype=int)
    for _ in range(num_simulations):
        grain_distribution = simulate_throw()
        total_grain_distribution += grain_distribution
    average_grain_distribution = total_grain_distribution / num_simulations
    len_distribution = 0
    while average_grain_distribution[len_distribution]:
        len_distribution += 1
    return average_grain_distribution[:len_distribution]

In [91]:
num_simulations = 100_000

In [93]:
average_grain_distribution = monte_carlo_throws_simulation(num_simulations)

In [94]:
print("ОТВЕТ:")
print("Среднее количество клеток с N зернами:")
for N, count in enumerate(average_grain_distribution):
    print(f"N = {N}: {count:.4f}")

ОТВЕТ:
Среднее количество клеток с N зернами:
N = 0: 23.3630
N = 1: 23.7286
N = 2: 11.8627
N = 3: 3.8861
N = 4: 0.9480
N = 5: 0.1791
N = 6: 0.0282
N = 7: 0.0037
N = 8: 0.0005
N = 9: 0.0000
N = 10: 0.0000


### **Задача 3. Порядок на экзамене**
Экзамен проходит по следующей схеме: если некоторый билет уже был
вытянут, то после ответа экзаменатор откладывает его в сторону. Студент
выучил 20 билетов из 30. Когда ему выгоднее идти - первым или вторым,
чтобы вероятность вынуть выученный билет была больше?

In [98]:
total_tickets = 30
learned_tickets = 20

#### 1) Напрямую вычисляем вероятности

In [101]:
# Вероятность вытянуть выученный билет, если идти первым
P1 = learned_tickets / total_tickets

In [103]:
# Вероятность вытянуть выученный билет, если идти вторым
P2_first_outcome = (learned_tickets - 1) / (total_tickets - 1) * P1 # если первый студент вытянул выученный билет
P2_second_outcome = learned_tickets / (total_tickets - 1) * (1 - P1) # если первый студент вытянул невыученный билет
P2 = P2_first_outcome + P2_second_outcome

In [105]:
print(f"Если студент пошел отвечать первым, вероятность вытянуть выученный билет - {P1:.4f}")
print(f"Если студент пошел отвечать вторым, вероятность вытянуть выученный билет - {P2:.4f}")

Если студент пошел отвечать первым, вероятность вытянуть выученный билет - 0.6667
Если студент пошел отвечать вторым, вероятность вытянуть выученный билет - 0.6667


##### ОТВЕТ: Студенту безразлично, идти первым или вторым, так как вероятности вытянуть выученный билет в обоих случаях равны.

#### 2) Симуляция методом Монте-Карло

In [125]:
def monte_carlo_exam_simulation(num_simulations):
    """Проводит серию симуляций экзамена методом Монте-Карло"""
    success_first = 0
    success_second = 0
    for _ in range(num_simulations):
        tickets = [True] * learned_tickets + [False] * (total_tickets - learned_tickets)
        random.shuffle(tickets)

        first_ticket = tickets.pop()
        if first_ticket:
            success_first += 1
            
        second_ticket = tickets.pop()
        if second_ticket:
            success_second += 1

    P1 = success_first / num_simulations
    P2 = success_second / num_simulations

    return P1, P2

In [127]:
num_simulations = 100_000

In [129]:
P1, P2 = monte_carlo_exam_simulation(num_simulations)

In [130]:
print(f"Если студент пошел отвечать первым, вероятность вытянуть выученный билет - {P1:.4f}")
print(f"Если студент пошел отвечать вторым, вероятность вытянуть выученный билет - {P2:.4f}")

Если студент пошел отвечать первым, вероятность вытянуть выученный билет - 0.6672
Если студент пошел отвечать вторым, вероятность вытянуть выученный билет - 0.6683


##### ОТВЕТ: Студенту безразлично, идти первым или вторым, так как вероятности вытянуть выученный билет в обоих случаях равны.